In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gtfparse import read_gtf
from scipy.stats import fisher_exact
from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles

## Peaks for training chromBPNet
Peaks called for merged replicates BAM file with MACS2 p-value set to 0.01, filtered to exclude lenient allele-specific peaks with FDR<0.1

In [40]:
canonical_chroms = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17','chr18', 'chr19',
                    'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX']

for sample in ['NA12878', 'NA18983', 'HG01241', 'HG02601', 'HG03464']:
    narrow_peaks = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/peaks/{sample}_merged_peaks.narrowPeak',
                               sep = '\t', header = None)
    narrow_peaks = narrow_peaks[narrow_peaks[0].isin(canonical_chroms)]

    deseq_peaks = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/peaks/{sample}_peaks.canonical.replicated.no_blacklist.bed',
                              sep = '\t', header = None, names = ['seqname', 'start', 'end', 'peak_id'])

    deseq_res = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/asocr/asocr_{sample}.csv', sep = '\s+', header = 0)
    deseq_res['peak_id'] = deseq_res.index
    deseq_res = deseq_res.merge(deseq_peaks, on = 'peak_id')

    close_to_asocr = deseq_res[deseq_res['padj'] < 0.1]
    close_to_asocr_ids = close_to_asocr.merge(narrow_peaks, left_on = ['seqname', 'start', 'end'], right_on = [0, 1, 2])[3].values

    narrow_peaks[~narrow_peaks[3].isin(close_to_asocr_ids)].to_csv(f'/DATA/users/m.magnitov/hap_phen/chromBPNet/train_peaks/{sample}.pval_0.01.no_as_FDR_0.1.narrowPeak', sep = '\t', header = 0, index = 0)

## Peaks for predicting with chromBPNet
Replicated peaks (called for merged, then overlapped with rep1/rep2/rep3) with narrowPeaks parameters from calling on merged with MACS2 p-value set to 0.01, lifted over to hap1 and hap2 coordinates

In [39]:
for sample in ['NA12878', 'NA18983', 'HG01241', 'HG02601', 'HG03464']:
    narrow_peaks = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/peaks/{sample}_merged_peaks.narrowPeak',
                               sep = '\t', header = None)
    
    deseq_peaks = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/peaks/{sample}_peaks.canonical.replicated.no_blacklist.bed',
                              sep = '\t', header = None)
    
    hap1_peaks = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/peaks/{sample}_peaks_hap1.bed', sep = '\t', header = None)
    hap2_peaks = pd.read_csv(f'/DATA/users/m.magnitov/hap_phen/ATACseq/peaks/{sample}_peaks_hap2.bed', sep = '\t', header = None)
    
    merged_peaks_trace = deseq_peaks.merge(hap1_peaks, on = [0, 3]).merge(hap2_peaks, on = [0, 3])
    merged_peaks_trace.columns = np.arange(len(merged_peaks_trace.columns))
    merged_peaks_trace = merged_peaks_trace.merge(narrow_peaks, on = [0, 1, 2])
    merged_peaks_trace.columns = ['chrom', 'start_hg38', 'end_hg38', 'peak_id_replicated',
                                  'start_hap1', 'end_hap1', 'start_hap2', 'end_hap2', 'peak_id', 4, 5, 6, 7, 8, 9]

    merged_peaks_trace[['chrom', 'start_hap1', 'end_hap1', 'peak_id', 4, 5, 6, 7, 8, 9]].to_csv(f'/DATA/users/m.magnitov/hap_phen/chromBPNet/predict_peaks/{sample}_hap1.narrowPeak', sep = '\t', header = 0, index = 0)
    merged_peaks_trace[['chrom', 'start_hap2', 'end_hap2', 'peak_id', 4, 5, 6, 7, 8, 9]].to_csv(f'/DATA/users/m.magnitov/hap_phen/chromBPNet/predict_peaks/{sample}_hap2.narrowPeak', sep = '\t', header = 0, index = 0)
    merged_peaks_trace[['chrom', 'start_hg38', 'end_hg38', 'peak_id', 4, 5, 6, 7, 8, 9]].to_csv(f'/DATA/users/m.magnitov/hap_phen/chromBPNet/predict_peaks/{sample}_hg38.narrowPeak', sep = '\t', header = 0, index = 0)